In [ ]:
from langgraph.graph import StateGraph, END
from modules.agent_state import AgentState
from modules.nodes import Nodes  # Import the Nodes class

# StateGraph 
builder = StateGraph(AgentState)
builder.add_node("user_input", Nodes.user_input_node)
builder.add_node("retrieve", Nodes.retrieve_node)
builder.add_node("generate_response", Nodes.generate_response_node)
builder.add_node("update_memory", Nodes.update_memory_node)

builder.set_entry_point("user_input")
builder.add_edge("user_input", "retrieve")
builder.add_edge("retrieve", "generate_response")
builder.add_edge("generate_response", "update_memory")
builder.add_edge("update_memory", END)

rag_graph = builder.compile()


#Client UI

import gradio as gr
from langchain.memory import ConversationBufferMemory
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.llms import Ollama

# Mevcut vektör veritabanının adı
DB_NAME = "vector-db"

# Vektör veritabanını yükleme
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings)
retriever = vectorstore.as_retriever()

# Ollama modelini başlat
llm = Ollama(model="llama3.2")

# Chat Memory (Konuşma geçmişi için)
memory = ConversationBufferMemory()

def query_rag_pipeline(user_query, history):
    # Konuşma geçmişini al
    chat_history = memory.load_memory_variables({}).get("history", "")
    
    # RAG ile en alakalı belgeleri al
    retrieved_docs = vectorstore.similarity_search(user_query, k=10)
    combined_context = " ".join([doc.page_content for doc in retrieved_docs])

    # Prompt formatı
    prompt = f"""
    You are a telecom assistant. Your answers should be based on the context and chat history provided. If the context is not relevant to the user's query, politely state that you do not have the required information.

    Chat History: {chat_history}
    
    Context: {combined_context}
    
    Question: {user_query}
    
    Answer:
    """

    response = llm.invoke(prompt)

    # Konuşma geçmişini güncelle
    memory.save_context({"input": user_query}, {"output": response})
    
    return response

# Gradio UI
gr.ChatInterface(fn=query_rag_pipeline, title="Chatbot RAG Assistant 🤖", type="messages").launch(share=True)


In [ ]:
from langgraph.graph import StateGraph, END
from modules.agent_state import AgentState
from modules.nodes import Nodes  # Import the Nodes class
from helpers.config import embeddings, vectorstore, retriever, llm, memory  # Import the configuration

# StateGraph 
builder = StateGraph(AgentState)
builder.add_node("user_input", Nodes.user_input_node)
builder.add_node("retrieve", Nodes.retrieve_node)
builder.add_node("generate_response", Nodes.generate_response_node)
builder.add_node("update_memory", Nodes.update_memory_node)

builder.set_entry_point("user_input")
builder.add_edge("user_input", "retrieve")
builder.add_edge("retrieve", "generate_response")
builder.add_edge("generate_response", "update_memory")
builder.add_edge("update_memory", "user_input")  # Loop back to user_input instead of END

rag_graph = builder.compile()
#Client UI

import gradio as gr
from langchain.memory import ConversationBufferMemory
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_community.llms import Ollama

# Mevcut vektör veritabanının adı
DB_NAME = "vector-db"


def query_rag_pipeline(user_query, history):
    # Initialize the state
    state = AgentState()
    state["user_query"] = user_query
    
    # Process the query through the StateGraph
    rag_graph.run(state)
    
    # Get the response from the state
    response = state.get("response", "Sorry, I couldn't generate a response.")
    
    return response

# Gradio UI
gr.ChatInterface(fn=query_rag_pipeline, title="Chatbot RAG Assistant 🤖", type="messages").launch(share=True)

Running on local URL:  http://127.0.0.1:7861

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/02/25 18:02:09 [W] [service.go:132] login to server failed: i/o deadline reached


Traceback (most recent call last):
  File "c:\Users\CAN\anaconda3\envs\LangChainEnv\lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\CAN\anaconda3\envs\LangChainEnv\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\CAN\anaconda3\envs\LangChainEnv\lib\site-packages\gradio\blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "c:\Users\CAN\anaconda3\envs\LangChainEnv\lib\site-packages\gradio\blocks.py", line 1518, in call_function
    prediction = await fn(*processed_input)
  File "c:\Users\CAN\anaconda3\envs\LangChainEnv\lib\site-packages\gradio\utils.py", line 793, in async_wrapper
    response = await f(*args, **kwargs)
  File "c:\Users\CAN\anaconda3\envs\LangChainEnv\lib\site-packages\gradio\chat_interface.py", line 623, in _submit_fn
    response = await anyio.to_thread.run_sync(
  

In [10]:
from langgraph.graph import StateGraph, END
from modules.agent_state import AgentState
from modules.nodes import Nodes  # Import the Nodes class
from helpers.config import embeddings, vectorstore, retriever, llm, memory  # Import the configuration

# StateGraph 
builder = StateGraph(AgentState)
builder.add_node("user_input", Nodes.user_input_node)
builder.add_node("retrieve", Nodes.retrieve_node)
builder.add_node("generate_response", Nodes.generate_response_node)
builder.add_node("update_memory", Nodes.update_memory_node)

builder.set_entry_point("user_input")
builder.add_edge("user_input", "retrieve")
builder.add_edge("retrieve", "generate_response")
builder.add_edge("generate_response", "update_memory")
builder.add_edge("update_memory", "user_input")  # Loop back to user_input instead of END

rag_graph = builder.compile()

# Client UI

import gradio as gr
def query_rag_pipeline(user_query, history):
    # Initialize the state
    state = AgentState()
    state["user_query"] = user_query
    
    # Process the query through the StateGraph
    for _ in rag_graph.stream(state):
        pass
    
    # Get the response from the state
    response = state.get("response", "Sorry, I couldn't generate a response.")
    
    return response

# Gradio UI

gr.ChatInterface(fn=query_rag_pipeline, title="Chatbot RAG Assistant 🤖", type="messages").launch(share=True)

Running on local URL:  http://127.0.0.1:7863

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/02/25 18:07:11 [W] [service.go:132] login to server failed: i/o deadline reached


Traceback (most recent call last):
  File "c:\Users\CAN\anaconda3\envs\LangChainEnv\lib\site-packages\gradio\queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "c:\Users\CAN\anaconda3\envs\LangChainEnv\lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "c:\Users\CAN\anaconda3\envs\LangChainEnv\lib\site-packages\gradio\blocks.py", line 1935, in process_api
    result = await self.call_function(
  File "c:\Users\CAN\anaconda3\envs\LangChainEnv\lib\site-packages\gradio\blocks.py", line 1518, in call_function
    prediction = await fn(*processed_input)
  File "c:\Users\CAN\anaconda3\envs\LangChainEnv\lib\site-packages\gradio\utils.py", line 793, in async_wrapper
    response = await f(*args, **kwargs)
  File "c:\Users\CAN\anaconda3\envs\LangChainEnv\lib\site-packages\gradio\chat_interface.py", line 623, in _submit_fn
    response = await anyio.to_thread.run_sync(
  

In [5]:
# Örnek ççalıştırmak için
test_input = {"user_query": "What is 5G Network slicing?"}
for s in rag_graph.stream(test_input):
    print(s)

{'user_input': {'user_query': 'What is 5G Network slicing?'}}
{'retrieve': {'retrieved_docs': ['slicing for virtualized beyond 5g networks,” Computer Networks, vol. 247, p. 110445, 2024.\n[512] N. Makris, V. Passas, A. Apostolaras, T. Tsourdinis, I. Chatzistefanidis, and T. Korakis, “On enabling\nremote hands-on computer networking education: the nitos testbed approach,” in 2023 IEEE Integrated\nSTEM Education Conference (ISEC).\nIEEE, 2023, pp. 132–138.\n[513] I. Chatzistefanidis, N. Makris, V. Passas, and T. Korakis, “Ue statistics time-series (cqi) in lte networks,”\n2022.\n[514] T. Tsourdinis, I. Chatzistefanidis, N. Makris, and T. Korakis, “Ue network traffic time-series (applications,\nthroughput, latency, cqi) in lte/5g networks,” IEEE Dataport, 2022.\n[515] D. Spathis and F. Kawsar, “The first step is the hardest: Pitfalls of representing and tokenizing temporal\ndata for large language models,” Journal of the American Medical Informatics Association, vol. 31, no. 9,\npp. 2151–

## stategraph with grader

In [11]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List
from langchain.memory import ConversationBufferMemory
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama

# Grader için Prompt
GRADER_PROMPT_TEMPLATE = """
You are an expert evaluator. Grade the response based on the following criteria:
1. Groundedness: Does the response fully address the question using the provided context?
2. Answer Relevance: Is the response relevant and directly answering the question?
3. Context Relevance: Is the response actually correct based on the context?

Context:
{context}

Question:
{question}

Response:
{response}

---

Provide a float score from 0 to 1 for each criteria and a short explanation for the score.
"""

class Grader:
    def __init__(self):
        self.model = Ollama(model="llama3.2")  # Grading LLM

    def grade(self, context: str, question: str, response: str) -> str:
        prompt_template = ChatPromptTemplate.from_template(GRADER_PROMPT_TEMPLATE)
        prompt = prompt_template.format(
            context=context, question=question, response=response
        )
        return self.model.invoke(prompt)

# AgentState yapısı
class AgentState(TypedDict):
    user_query: str
    retrieved_docs: List[str]
    response: str
    chat_history: str
    evaluation_score: float

# Hafıza ve vektör veritabanı yükleme
DB_NAME = "vector-db"
embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings)
retriever = vectorstore.as_retriever()
llm = Ollama(model="llama3.2")
memory = ConversationBufferMemory()
grader = Grader()

# Düğümler (Nodes)
def user_input_node(state: AgentState):
    return {"user_query": state["user_query"]}

def retrieve_node(state: AgentState):
    retrieved_docs = vectorstore.similarity_search(state["user_query"], k=10)
    return {"retrieved_docs": [doc.page_content for doc in retrieved_docs]}

def generate_response_node(state: AgentState):
    combined_context = "\n".join(state["retrieved_docs"])
    prompt = f"""
    You are a telecom assistant. Your answers should be based on the retrieved context below.
    
    Context: {combined_context}
    
    Question: {state['user_query']}
    
    Answer:
    """
    response = llm.invoke(prompt)
    return {"response": response}

def evaluate_response_node(state: AgentState):
    evaluation_result = grader.grade(
        context="\n".join(state["retrieved_docs"]),
        question=state["user_query"],
        response=state["response"]
    )
    
    print("Grader Output:", evaluation_result)  # Debug için çıktıyı yazdır
    
    score_values = []
    score_lines = evaluation_result.split("\n")
    
    for line in score_lines:
        if any(keyword in line for keyword in ["Overall Score", "Final Score", "Total Score"]):
            try:
                score = float(line.split(":")[1].strip().replace('*', ''))
                score_values.append(score)
            except (IndexError, ValueError):
                continue
    
    final_score = max(score_values, default=0.0)  # Eğer hiç uygun skor bulunmazsa 0.0 ata
    return {"evaluation_score": final_score}

def revise_response_node(state: AgentState):
    if state["evaluation_score"] < 0.5:
        return {"response": generate_response_node(state)["response"]}
    return {}

def update_memory_node(state: AgentState):
    memory.save_context({"input": state["user_query"]}, {"output": state["response"]})
    return {"chat_history": memory.load_memory_variables({}).get("history", "")}

# StateGraph oluşturma
builder = StateGraph(AgentState)
builder.add_node("user_input", user_input_node)
builder.add_node("retrieve", retrieve_node)
builder.add_node("generate_response", generate_response_node)
builder.add_node("evaluate_response", evaluate_response_node)
builder.add_node("revise_response", revise_response_node)
builder.add_node("update_memory", update_memory_node)

# İş akışını belirleme
builder.set_entry_point("user_input")
builder.add_edge("user_input", "retrieve")
builder.add_edge("retrieve", "generate_response")
builder.add_edge("generate_response", "evaluate_response")
builder.add_conditional_edges(
    "evaluate_response", 
    lambda state: "revise_response" if state["evaluation_score"] < 0.5 else "update_memory",
    {"revise_response": "revise_response", "update_memory": "update_memory"}
)
builder.add_edge("revise_response", "generate_response")
builder.add_edge("update_memory", END)

# Graph oluştur
rag_graph = builder.compile()

# Örnek çalıştırma
test_input = {"user_query": "What is 5G Network slicing?"}
for s in rag_graph.stream(test_input):
    print(s)


{'user_input': {'user_query': 'What is 5G Network slicing?'}}
{'retrieve': {'retrieved_docs': ['slicing for virtualized beyond 5g networks,” Computer Networks, vol. 247, p. 110445, 2024.\n[512] N. Makris, V. Passas, A. Apostolaras, T. Tsourdinis, I. Chatzistefanidis, and T. Korakis, “On enabling\nremote hands-on computer networking education: the nitos testbed approach,” in 2023 IEEE Integrated\nSTEM Education Conference (ISEC).\nIEEE, 2023, pp. 132–138.\n[513] I. Chatzistefanidis, N. Makris, V. Passas, and T. Korakis, “Ue statistics time-series (cqi) in lte networks,”\n2022.\n[514] T. Tsourdinis, I. Chatzistefanidis, N. Makris, and T. Korakis, “Ue network traffic time-series (applications,\nthroughput, latency, cqi) in lte/5g networks,” IEEE Dataport, 2022.\n[515] D. Spathis and F. Kawsar, “The first step is the hardest: Pitfalls of representing and tokenizing temporal\ndata for large language models,” Journal of the American Medical Informatics Association, vol. 31, no. 9,\npp. 2151–

KeyboardInterrupt: 

In [4]:
# Örnek çalıştırma
test_input = {"user_query": "What is 5G Network slicing?"}
for s in rag_graph.stream(test_input):
    print(s)

{'user_input': {'user_query': 'What is 5G Network slicing?'}}
{'retrieve': {'retrieved_docs': ['slicing for virtualized beyond 5g networks,” Computer Networks, vol. 247, p. 110445, 2024.\n[512] N. Makris, V. Passas, A. Apostolaras, T. Tsourdinis, I. Chatzistefanidis, and T. Korakis, “On enabling\nremote hands-on computer networking education: the nitos testbed approach,” in 2023 IEEE Integrated\nSTEM Education Conference (ISEC).\nIEEE, 2023, pp. 132–138.\n[513] I. Chatzistefanidis, N. Makris, V. Passas, and T. Korakis, “Ue statistics time-series (cqi) in lte networks,”\n2022.\n[514] T. Tsourdinis, I. Chatzistefanidis, N. Makris, and T. Korakis, “Ue network traffic time-series (applications,\nthroughput, latency, cqi) in lte/5g networks,” IEEE Dataport, 2022.\n[515] D. Spathis and F. Kawsar, “The first step is the hardest: Pitfalls of representing and tokenizing temporal\ndata for large language models,” Journal of the American Medical Informatics Association, vol. 31, no. 9,\npp. 2151–

IndexError: list index out of range